# Before Running the Script Please Make Sure you Select the Run Time as GPU

#**Clone the Github Repo**

In [1]:
import os
print("Current Working Directory:", os.getcwd())


Current Working Directory: E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\YOLOv8-DeepSORT-Object-Tracking


In [2]:
import os
#os.chdir(r"E:\school\YoloDeepSort\YOLOv8-DeepSORT\content\YOLOv8-DeepSORT-Object-Tracking")
print("Current Working Directory:", os.getcwd())  

os.chdir(os.path.join(os.getcwd(), "content", "YOLoV8-DeepSORT-Object-Tracking"))


Current Working Directory: E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\YOLOv8-DeepSORT-Object-Tracking


FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'E:\\School\\YoloDeepSort\\YOLOv8-DeepSORT\\content\\YOLOv8-DeepSORT-Object-Tracking\\content\\YOLoV8-DeepSORT-Object-Tracking'

In [2]:
import torch
print("CUDA 是否可用:", torch.cuda.is_available())
print("CUDA 版本:", torch.version.cuda)
print("当前 GPU 数量:", torch.cuda.device_count())


CUDA 是否可用: True
CUDA 版本: 12.4
当前 GPU 数量: 1


##Setting the Current Directory

In [3]:
%cd E:\school\YoloDeepSort\YOLOv8-DeepSORT

E:\school\YoloDeepSort\YOLOv8-DeepSORT


In [4]:
%cd content\YOLOv8-DeepSORT-Object-Tracking\ultralytics\yolo\v8\detect

E:\school\YoloDeepSort\YOLOv8-DeepSORT\content\YOLOv8-DeepSORT-Object-Tracking\ultralytics\yolo\v8\detect


In [ ]:
%cd content\YOLOv8-DeepSORT-Object-Tracking
!pip install -e '.[dev]'
#这些都在控制台执行比较好点，用于安装依赖

In [10]:
import ultralytics
print(ultralytics.__version__)

8.0.3


##Lets Run the Script 运行预测脚本

In [9]:
!python predict_customer.py source="test2.mp4" project="E:/School/YoloDeepSort/YOLOv8-DeepSORT/content" name="customerTest"
#write 


[2025-03-14 19:34:39,908][root.tracker][INFO] - Loading weights from deep_sort_pytorch/deep_sort/deep/checkpoint/ckpt.t7... Done!


E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\YOLOv8-DeepSORT-Object-Tracking\ultralytics\yolo\v8\detect\deep_sort_pytorch\deep_sort\deep\feature_extractor.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues rel

In [15]:
import cv2
import os

# 确保路径正确
seq_file = r"E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\output\track_1\1.seq"
track_folder = os.path.dirname(seq_file)

# 读取 .seq 文件的前几行
with open(seq_file, 'r') as f:
    lines = f.readlines()

print("=== .SEQ 文件内容（前5行） ===")
for line in lines[:5]:  # 只显示前5行，避免太长
    print(line.strip())

# 检查第一帧图片路径
first_frame_idx = lines[0].split(',')[0]
img_path = os.path.join(track_folder, f"1_{first_frame_idx}.jpg")

print(f"\n=== 检查第一帧图片: {img_path} ===")
if os.path.exists(img_path):
    img = cv2.imread(img_path)
    if img is not None:
        print(f"✅ 读取成功, 尺寸: {img.shape}")
    else:
        print("❌ 读取失败 (可能图片损坏)")
else:
    print("❌ 图片不存在")


=== .SEQ 文件内容（前5行） ===
18,269,366,601,1279
19,256,375,594,1279
20,244,385,585,1279
21,234,396,577,1279
22,225,405,568,1279

=== 检查第一帧图片: E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\output\track_1\1_18.jpg ===
✅ 读取成功, 尺寸: (913, 332, 3)


In [ ]:
video_writer.release()


In [33]:
import cv2
import os

def seq_to_video(seq_file, output_dir, video_fps=0.1):
    """
    读取 .seq 轨迹文件，并将对应裁剪的目标图片合成视频。
    :param seq_file: .seq 文件路径
    :param output_dir: 目标裁剪图片所在目录
    :param video_fps: 视频帧率 (默认10, 防止视频过短)
    """
    track_id = os.path.basename(seq_file).split(".")[0]  # 获取 track_id
    track_folder = os.path.dirname(seq_file)  # 目标图片存放路径

    if not os.path.exists(seq_file):
        print(f"[Error] .seq 文件不存在: {seq_file}")
        return

    # 读取 .seq 文件，按帧编号排序
    frames_info = []
    with open(seq_file, 'r') as f:
        for line in f:
            data = line.strip().split(',')
            if len(data) < 5:
                print(f"[Warning] 数据格式错误，跳过该行: {line.strip()}")
                continue

            try:
                frame_idx = int(data[0])  # 帧编号
                x1, y1, x2, y2 = map(int, data[1:5])  # 获取 bbox
                frames_info.append((frame_idx, x1, y1, x2, y2))
            except ValueError:
                print(f"[Warning] 数值解析错误，跳过该行: {line.strip()}")

    if not frames_info:
        print("[Error] 没有可用的帧数据，无法生成视频")
        return

    frames_info.sort(key=lambda x: x[0])

    # 生成完整帧序列，填充缺失帧
    all_expected_frames = list(range(frames_info[0][0], frames_info[-1][0] + 1))
    existing_frames = {frame[0] for frame in frames_info}
    missing_frames = sorted(set(all_expected_frames) - existing_frames)
    print(f"[Info] 发现 {len(missing_frames)} 帧缺失，自动填充")

    # 获取第一帧确定尺寸
    first_frame_idx, x1, y1, x2, y2 = frames_info[0]
    first_img_path = os.path.join(track_folder, f"1_{first_frame_idx}.jpg")

    if not os.path.exists(first_img_path):
        print(f"[Error] 找不到图片: {first_img_path}")
        return

    first_frame = cv2.imread(first_img_path)
    if first_frame is None:
        print(f"[Error] 第一帧图片损坏: {first_img_path}")
        return

    height, width, _ = first_frame.shape

    # 设置视频编码器（尝试更兼容的格式）
    video_path = os.path.join(output_dir, f"track_{track_id}.mp4")
    fourcc = cv2.VideoWriter_fourcc(*'XVID')  # 试试 XVID 编码
    video_writer = cv2.VideoWriter(video_path, fourcc, video_fps, (width, height))

    if not video_writer.isOpened():
        print("[Error] 无法创建视频文件")
        return

    last_valid_frame = None
    frame_count = 0  # 记录写入的帧数
    
    for frame_idx in all_expected_frames:
        img_path = os.path.join(track_folder, f"1_{frame_idx}.jpg")
    
        if not os.path.exists(img_path):
            print(f"[Warning] 缺失 {img_path}, 用上一帧填充")
            if last_valid_frame is not None:
                video_writer.write(last_valid_frame)
                frame_count += 1
            continue
    
        frame = cv2.imread(img_path)
        if frame is None or frame.shape[0] == 0 or frame.shape[1] == 0:
            print(f"[Error] 读取失败: {img_path}")
            continue
    
        if frame.mean() == 0:
            print(f"[Error] 发现空白帧: {img_path}, 跳过")
            continue
    
        last_valid_frame = frame.copy()
        video_writer.write(frame)
        frame_count += 1
        print(f"[✔] 写入帧 {frame_idx}: {img_path}")  # 确保写入

    video_writer.release()
    print(f"[✔] 视频生成完成: {video_path}, 总帧数: {frame_count}")

# 使用示例
seq_file = r"E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\output\track_1\1.seq"
output_dir = r"E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\output"
seq_to_video(seq_file, output_dir)


[Info] 发现 226 帧缺失，自动填充
[✔] 写入帧 18: E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\output\track_1\1_18.jpg
[✔] 写入帧 19: E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\output\track_1\1_19.jpg
[✔] 写入帧 20: E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\output\track_1\1_20.jpg
[✔] 写入帧 21: E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\output\track_1\1_21.jpg
[✔] 写入帧 22: E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\output\track_1\1_22.jpg
[✔] 写入帧 23: E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\output\track_1\1_23.jpg
[✔] 写入帧 24: E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\output\track_1\1_24.jpg
[✔] 写入帧 25: E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\output\track_1\1_25.jpg
[✔] 写入帧 26: E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\output\track_1\1_26.jpg
[✔] 写入帧 27: E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\output\track_1\1_27.jpg
[✔] 写入帧 28: E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\output\track_1\1_28.jpg
[✔] 写入帧 29: E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\outpu

In [34]:
import imageio_ffmpeg as ffmpeg
import subprocess

video_path = r"E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\output\track_1.mp4"

cmd = [ffmpeg.get_ffmpeg_exe(), "-i", video_path, "-hide_banner"]
process = subprocess.run(cmd, capture_output=True, text=True)

print(process.stderr)  # 这里会输出 `.mp4` 的真实帧数


Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\output\track_1.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2mp41
    encoder         : Lavf58.76.100
  Duration: 00:00:10.00, start: 0.000000, bitrate: 12 kb/s
  Stream #0:0[0x1](und): Video: mpeg4 (Simple Profile) (mp4v / 0x7634706D), yuv420p, 332x912 [SAR 1:1 DAR 83:228], 11 kb/s, 0.10 fps, 0.10 tbr, 16384 tbn (default)
      Metadata:
        handler_name    : VideoHandler
        vendor_id       : [0][0][0][0]
At least one output file must be specified



In [30]:
!ffmpeg -i track_1.mp4 -vcodec libx264 -crf 23 -r 10 output_fixed.mp4

'ffmpeg' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [21]:
!python predict.py source="test1.mp4" 

save_dir is not a valid key. Similar keys: ['save_crop', 'save', 'save_hybrid']


In [21]:
!python predict.py source="test1.mp4" project="E:/School/YoloDeepSort/YOLOv8-DeepSORT/content" name="my_results"


[2025-03-10 15:57:12,927][root.tracker][INFO] - Loading weights from deep_sort_pytorch/deep_sort/deep/checkpoint/ckpt.t7... Done!


E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\YOLOv8-DeepSORT-Object-Tracking\ultralytics\yolo\v8\detect\deep_sort_pytorch\deep_sort\deep\feature_extractor.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues rel

##Display the Demo Video

In [22]:
import imageio_ffmpeg
import subprocess

# 获取 ffmpeg 可执行文件路径
ffmpeg_path = imageio_ffmpeg.get_ffmpeg_exe()

save_path = r"E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\YOLOv8-DeepSORT-Object-Tracking\runs\detect\train7\test1.mp4"
compressed_path = r"E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\result_compressed1.mp4"

ffmpeg_command = f'"{ffmpeg_path}" -i "{save_path}" -vcodec libx264 "{compressed_path}"'
print("正在执行:", ffmpeg_command)

# 运行 ffmpeg
result = subprocess.run(ffmpeg_command, shell=True, capture_output=True, text=True)

# 打印输出
print("标准输出:", result.stdout)
print("错误信息:", result.stderr)


正在执行: "F:\anoconda\envs\YOLOv8-DeepSORT\Lib\site-packages\imageio_ffmpeg\binaries\ffmpeg-win-x86_64-v7.1.exe" -i "E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\YOLOv8-DeepSORT-Object-Tracking\runs\detect\train7\test1.mp4" -vcodec libx264 "E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\result_compressed1.mp4"
标准输出: 
错误信息: ffmpeg version 7.1-essentials_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-libfreetype --enable-libfribidi --enable-libh

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
save_path = r"E:\学校\YoloDeepSort\YOLOv8-DeepSORT\content\YOLOv8-DeepSORT-Object-Tracking\runs\detect\train4\test1.mp4"

# Compressed video path
compressed_path = r"E:\学校\YoloDeepSort\YOLOv8-DeepSORT\content\result_compressed.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#查看追踪的视频
import cv2

video_path = r"E:\学校\YoloDeepSort\YOLOv8-DeepSORT\content\result_compressed.mp4"

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("⚠️ 无法打开视频文件！")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break  # 读取完毕
    cv2.imshow("Video Playback", frame)

    # 按 `q` 退出播放
    if cv2.waitKey(25) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
